In [1]:
import random

In [2]:
random.seed(0)

In [1]:
from pynvml import *
nvmlInit()
handle = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(handle)
print("Used memory: {0}%".format(int(100*info.used/info.total)))


Used memory: 17%


In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
import warnings
warnings.filterwarnings("ignore")
marks = pd.read_csv('D:/Kaggle/train_ship_segmentations.csv') # Markers for ships
images = os.listdir('D:/Kaggle/train_images') # Images for training
os.chdir("D:/Kaggle/train_images")

In [5]:
# from pynvml import *
nvmlInit()
handle = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(handle)
print("Used memory: {0}%".format(int(100*info.used/info.total)))


Used memory: 1%


In [6]:
from sys import getsizeof as size 

In [8]:
from skimage import io
from skimage import measure 
from skimage import feature
from skimage.morphology import disk
from skimage.filters import rank
from skimage import exposure

In [9]:
# test = pd.read_csv('../COMPETITION/test_ship_segmentations.csv')
# test_images = os.listdir('../COMPETITION/test_images')

In [10]:
# test["ImageId"].unique().shape[0] == len(test_images)
# len(test_images)

In [11]:
marks['ships'] = marks['EncodedPixels'].map(lambda c_row: 1 if isinstance(c_row, str) else 0)
# unique_img_ids = marks.groupby('ImageId').agg({'ships': 'sum'}).reset_index()

In [12]:
marks[marks["ImageId"] == "3b4251e3c.jpg"]

,ImageId,EncodedPixels,ships
30205,3b4251e3c.jpg,204044 1 204811 3 205578 6 206346 7 207113 9 2...,1
30206,3b4251e3c.jpg,271802 2 272568 4 273334 7 274099 10 274865 13...,1


In [13]:
marks.head()

,ImageId,EncodedPixels,ships
0,00003e153.jpg,NaN,0
1,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...,1
2,00021ddc3.jpg,101361 1 102128 3 102896 4 103663 6 104430 9 1...,1
3,00021ddc3.jpg,95225 2 95992 5 96760 7 97527 9 98294 9 99062 ...,1
4,00021ddc3.jpg,74444 4 75212 4 75980 4 76748 4 77517 3 78285 ...,1


In [14]:
train_data = marks[marks["ships"]!=0]
train_ids = list(train_data["ImageId"])
no_ships_train = list(set(images) & set(train_ids))
train_images_ids = list(set(no_ships_train) & set(images))

In [15]:
train_data.shape

(56030, 3)

In [16]:
def Equalizer(file):
    '''
    Equalizes image spectrum (in gray scale)
    '''
    img = io.imread(file, as_grey=True)
    selem = disk(100)
    img_eq = rank.equalize(img, selem=selem)
    return img_eq

In [17]:
def mask_part(pic):
    '''
    Function that encodes mask for single ship from .csv entry into numpy matrix
    '''
    back = np.zeros(768**2)
    starts = pic.split()[0::2]
    lens = pic.split()[1::2]
    for i in range(len(lens)):
        back[(int(starts[i])-1):(int(starts[i])-1+int(lens[i]))] = 1
    return np.reshape(back, (768, 768, 1))

def is_empty(key):
    '''
    Function that checks if there is a ship in image
    '''
    df = marks[marks['ImageId'] == key].iloc[:,1]
    if len(df) == 1 and type(df.iloc[0]) != str and np.isnan(df.iloc[0]):
        return True
    else:
        return False
    
def masks_all(key):
    '''
    Merges together all the ship markers corresponding to a single image
    '''
    df = marks[marks['ImageId'] == key].iloc[:,1]
    masks= np.zeros((768,768,1))
    if is_empty(key):
        return masks
    else:
        for i in range(len(df)):
            masks += mask_part(df.iloc[i])
        return np.transpose(masks, (1,0,2))

In [18]:
def transform(X, Y):
    '''
    Function for augmenting images. 
    It takes original image and corresponding mask and performs the
    same flipping and rotation transforamtions on both in order to 
    perserve the overlapping of ships and their masks
    '''
# add noise:
    
    x = np.copy(X)
    y = np.copy(Y)

    x[:,:,0] = x[:,:,0] + np.random.normal(loc=0.0, scale=0.01, size=(768,768))
    x[:,:,1] = x[:,:,1] + np.random.normal(loc=0.0, scale=0.01, size=(768,768))
    x[:,:,2] = x[:,:,2] + np.random.normal(loc=0.0, scale=0.01, size=(768,768))
    # Adding Gaussian noise on each rgb channel; this way we will NEVER get two completely same images.
    # Note that this transformation is not performed on Y 
    x[np.where(x<0)] = 0
    x[np.where(x>1)] = 1
# axes swap:
    if np.random.rand()<0.5: # 0.5 chances for this transformation to occur (same for two below)
        x = np.swapaxes(x, 0,1)
        y = np.swapaxes(y, 0,1)
# vertical flip:
    if np.random.rand()<0.5:
        x = np.flip(x, 0)
        y = np.flip(y, 0)
# horizontal flip:
    if np.random.rand()<0.5:
        x = np.flip(x, 1)
        y = np.flip(y, 1)
    return x, y  

In [19]:
def ship_gen(files):
    
    for id in files:
        yield id

In [20]:
def make_batch(files, batch_size):
    '''
    Creates batches of images and masks in order to feed them to NN
    '''
    X = np.zeros((batch_size, 768, 768, 3))
    Y = np.zeros((batch_size, 768, 768, 1)) # I add 1 here to get 4D batch
    for i in range(batch_size):
#         gen = ship_gen(files)
#         ship = next(gen)
        
        ship = np.random.choice(files)
#         print(ship)
        X[i] = (io.imread(ship))/255.0 # Original images are in 0-255 range, I want it in 0-1
        Y[i]= masks_all(ship)
    return X, Y

In [21]:
def Generator(files, batch_size):
    '''
    Generates batches of images and corresponding masks
    '''
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print("Used memory: {0}%".format(int(100*info.used/info.total)))
    while True:
        X, Y = make_batch(files, batch_size)
#         for i in range(batch_size):
#             X[i], Y[i] = transform(X[i], Y[i])
        yield X, Y

In [22]:
# Intersection over Union for Objects
def IoU(y_true, y_pred, tresh=1e-10):
    Intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    Union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - Intersection
    return K.mean( (Intersection + tresh) / (Union + tresh), axis=0)
# Intersection over Union for Background
def back_IoU(y_true, y_pred):
    return IoU(1-y_true, 1-y_pred)
# Loss function
def IoU_loss(in_gt, in_pred):
    
    #return 2 - back_IoU(in_gt, in_pred) - IoU(in_gt, in_pred)
    return 1 - IoU(in_gt, in_pred)

In [23]:
inputs = Input((768, 768, 3))

c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (inputs)
c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)

u5 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c4)
u5 = concatenate([u5, c3])
c5 = Conv2D(32, (3, 3), activation='relu', padding='same') (u5)
c5 = Conv2D(32, (3, 3), activation='relu', padding='same') (c5)

u6 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c2])
c6 = Conv2D(16, (3, 3), activation='relu', padding='same') (u6)
c6 = Conv2D(16, (3, 3), activation='relu', padding='same') (c6)

u7 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c1], axis=3)
c7 = Conv2D(8, (3, 3), activation='relu', padding='same') (u7)
c7 = Conv2D(8, (3, 3), activation='relu', padding='same') (c7)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c7)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss= IoU_loss, metrics=[IoU, back_IoU])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 768, 768, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 768, 768, 8)  224         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 768, 768, 8)  584         conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 384, 384, 8)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (

In [24]:
import gc; gc.enable()
gc.collect()

14

In [25]:
train_part = train_images_ids[:100]
len(train_part)

100

In [26]:
batch_size = 3
steps_per_epoch = int(train_data.shape[0]/(batch_size))+1
epochs = 10
print(steps_per_epoch)

18677


In [ ]:
model.load_weights('Unet_batch3_epoch_2_1.h5')

In [ ]:
results = model.fit_generator(Generator(train_images_ids, batch_size = 3), steps_per_epoch = steps_per_epoch, epochs = 10)

Epoch 1/10
Used memory: 1%
18677/18677 [==============================] - 8083s 433ms/step - loss: 0.9904 - IoU: 0.0096 - back_IoU: 0.0195
Epoch 2/10
18677/18677 [==============================] - 5857s 314ms/step - loss: 0.9964 - IoU: 0.0036 - back_IoU: 0.3618
Epoch 3/10
18677/18677 [==============================] - 5540s 297ms/step - loss: 1.0000 - IoU: 4.1793e-13 - back_IoU: 0.9949
Epoch 4/10
18677/18677 [==============================] - 5496s 294ms/step - loss: 1.0000 - IoU: 4.2498e-13 - back_IoU: 0.9949
Epoch 5/10
18677/18677 [==============================] - 5471s 293ms/step - loss: 1.0000 - IoU: 4.1631e-13 - back_IoU: 0.9949
Epoch 6/10
18677/18677 [==============================] - 5521s 296ms/step - loss: 1.0000 - IoU: 4.2306e-13 - back_IoU: 0.9949
Epoch 7/10
18677/18677 [==============================] - 5538s 297ms/step - loss: 1.0000 - IoU: 4.2586e-13 - back_IoU: 0.9949
Epoch 8/10
18071/18677 [============================>.] - ETA: 3:23 - loss: 1.0000 - IoU: 4.1724e-13 - 

In [ ]:
K.clear_session()

In [ ]:
model.save_weights('Unet_batch3_epoch_2_1.h5')

In [ ]:
os.chdir("../saved_models/")

In [ ]:
def Pred_Gen(images_data):
  
    for i in range(len(images_data)):
        yield io.imread(images_data[i]).reshape(1,768,768,3)

In [ ]:
g = triangleNums()


In [ ]:
print(d)

In [ ]:
os.chdir("../test_images/")
test_images = os.listdir('D:/Kaggle/test_images') 

In [ ]:
test_1_part = test_images[:10]

In [ ]:
g = Pred_Gen(test_1_part)


In [ ]:
X = np.zeros((10,768,768,3))
for i in range(10):
    X[i] = next(g) 

In [ ]:
res_1part = model.predict_generator(Pred_Gen(test_1_part),steps=len(test_1_part), workers=5)

In [ ]:
import gc; gc.enable()
gc.collect()

In [ ]:
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction